# Network Modifications - Roadways

In this notebook a copy of our baseline NetworkDataset is made to be modified reflecting the candidate improvement

In [ ]:
import arcpy
import os
import shutil
from arcgis.features import SpatialDataFrame
import pandas as pd

arcpy.CheckOutExtension("network")

base_path = os.path.abspath(".")
base_gdb = os.path.join(base_path, "nd.gdb")

target_gdb =  os.path.join(base_path, "mod_drive.gdb")

# if mod_drive.gdb exists, delete it
if os.path.isdir(target_gdb):
    shutil.rmtree(target_gdb)

arcpy.management.CreateFileGDB(base_path, "mod_drive")

arcpy.env.workspace = target_gdb

In [ ]:
# Copy our baseline network dataset to our dataset for modification
arcpy.management.Copy(
    os.path.join(base_gdb, r"NetworkDataset"), 
    os.path.join(target_gdb, "NetworkDataset")
)

In [ ]:
# if open in ArcGIS Pro, remove all layers
try:
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    mp = aprx.listMaps("Map")[0]
    for rmlyr in mp.listLayers():    
        if rmlyr.name not in ['World Topographic Map', 'World Hillshade']:        
            mp.removeLayer(rmlyr)
except OSError:
    pass

In [ ]:
# delete existing network
arcpy.management.Delete(os.path.join(target_gdb, r"NetworkDataset\NetworkDataset_ND"))

In [ ]:
arcpy.management.MakeFeatureLayer(
    os.path.join(target_gdb, r"NetworkDataset\BikePedAuto"), 
    "BPA"
)

# Make Edits

**Follow the instructions below for the appropriate section to make edits**


Project Type | Action 
---- | ----
New construction (line) | Add new line to network and make appropriate connections 
Widening (line) | Reduce travel time along links to free flow speed (Julie will ask Suzy what the travel time benefit is in the model) 
Operational (line) | Reduce travel time along links to average of free flow speed and peak hour speed 
Restripe (when is basically widening) (line) | Reduce travel time along links to free flow speed 
New interchange (point)  | Add connections in network 
Grade-separated crossing | Add new line across intersection (no intersection impedance) 



## New construction (line)
For new construction, new lines and connections are added to the network.

Note, when editing lines, take care to use snapping--overlapping verticies need to precisely connect.

Connecting the end point of a new line to the mid-point of an existint segment will not work. The existing segment must be split at the connection point.

Open the Edit menu bar.

1. Add new line to network (or copy existing line from source layer: Edit --> Copy --> Paste Special. Paste into BikePedAuto layer and do NOT include attributes.)
2. Edit vertices and ensure that beginning / end points connect to other beginning / end points. 
5. Update attributes of new feature (hint, use the Transfer Attributes tool to copy attributes from a similar nearby segment)

Suggested attribute values: 
* Oneway = B
* Speed (choose an appropriate speed based on functional class, etc. ped = 3 mph, bike = 11 mph)
* AutoNetwork = Y (or N for ped/transit only) -- (new transit service shouldn't be added to the BikePedAuto but to Transit Routes)
* BikeNetwork = Y
* Ped Network = Y
* DriveTime = set by script, calculated based on length (note: length is in meters) and speed
* hierarchy: 1 = Interstate 2 = Major Arterial 3 = Local
3. Select the intersecting road segments and use the Planarize Tool to split the intersecting lines (requires ArcGIS Pro Standard or Advanced). With a ArcGIS Pro License, use the Split tool.
6. Select all affected features and update length and speed attributes using cells below

In [ ]:
# UPDATE LENGTHS FOR SELECTED FEATURES - SELECT ONLY AFFECTED FEATURES TO KEEP RUN TIME REASONABLE
# this will likely throw some TypeErrors if the selectio includes non-roadway segments - ignore these!
if int(arcpy.management.GetCount("BikePedAuto")[0]) < 100:
    arcpy.management.CalculateField("BikePedAuto", "Length_Miles", '!shape.length@miles!', "PYTHON3", None, "DOUBLE")
    arcpy.management.CalculateField("BikePedAuto", "DriveTime", '!Length_Miles! / (!Speed! / 60)', "PYTHON3", None, "DOUBLE")
    arcpy.management.CalculateField("BikePedAuto", "PedestrianTime", '!Length_Miles! / (3 / 60)', "PYTHON3", None, "DOUBLE")
    arcpy.management.CalculateField("BikePedAuto", "BikeTime", '!Length_Miles! / (11 / 60)', "PYTHON3", None, "DOUBLE")
else:
    print("Warning: operation will affect more than 100 features - did you select only the intended target?")

## Widening (line)

For capacity expansion projects, the DriveTime attribute is reduced from the Peak Hour speed to the Free Flow speed (or by 20% when travel demand model travel times are not available).

Select segments to be modified

In [ ]:
# UPDATE LENGTHS FOR SELECTED FEATURES - SELECT ONLY AFFECTED FEATURES TO KEEP RUN TIME REASONABLE
# this will likely throw some TypeErrors if the selectio includes non-roadway segments - ignore these!
if int(arcpy.management.GetCount("BPA")[0]) < 100:
    arcpy.management.CalculateField("BPA", 
                                    "PK_SPD", '!FF_SPD!', "PYTHON3", None, "DOUBLE")
    arcpy.management.CalculateField("BPA", 
                                    "DriveTime_Peak", '!Length_Miles! / (!FF_SPD! / 60)', "PYTHON3", None, "DOUBLE")
    arcpy.management.CalculateField("BPA", "DriveTime", '!DriveTime_Peak!', "PYTHON3", None, "DOUBLE")
else:
    print("Warning: operation will affect more than 100 features - did you select only the intended target?")

## Operational (line)

## Restripe (when is basically widening) (line) 
Reduce travel time along links to free flow speed 


## New interchange (point) 
Add connections in network 
Grade-separated crossing | Add new line across intersection (no intersection impedance) 

# Save Edits

In [ ]:
# don't forget to clear the selection before creating the new network d"ataset
arcpy.management.SelectLayerByAttribute("BPA", "CLEAR_SELECTION")

**save edits!**

Save edits to BikePedAuto layer and remove from map. Then run the cells below to create the network dataset and build it.

In [ ]:
# create network dataset from template
arcpy.na.CreateNetworkDatasetFromTemplate(
    os.path.join(base_path, "template.xml"),                 
    os.path.join(target_gdb, "NetworkDataset")
)

Set up driving mode if necessary.

In [ ]:
# finally, build the dataset
arcpy.na.BuildNetwork(os.path.join(target_gdb, r"NetworkDataset\NetworkDataset_ND"))